In [30]:
! pip install -qU wikipedia python-dotenv 'crewai[tools]'

In [22]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun, Tool, StructuredTool
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import BSHTMLLoader
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
#initializing LLM model
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Tools

In [26]:
# Initialize existing tools
search_tool = SerperDevTool()
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia = WikipediaAPIWrapper()

def wikipedia_search(query):
    return wikipedia.run(query)

wikipedia_tool = Tool(
    name="WikipediaSearch",
    func=wikipedia_search,
    description="Search Wikipedia for information on a given topic. Input should be a search query."
)

web_scraper_tool = ScrapeWebsiteTool()

In [27]:
# Citation Manager Tool
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

# Define a Pydantic model for the citation input
class CitationInput(BaseModel):
    title: str = Field(description="The title of the work")
    authors: str = Field(description="The authors of the work")
    year: str = Field(description="The year of publication")
    source: str = Field(description="The source or publisher of the work")

# Define the citation generation function
def generate_citation(input_data: CitationInput) -> str:
    return f"{input_data.authors}. ({input_data.year}). {input_data.title}. {input_data.source}."

# Create the StructuredTool for citation management
citation_tool = StructuredTool.from_function(
    func=generate_citation,
    name="CitationManager",
    description="Generate a citation given the title, authors, year, and source",
    args_schema=CitationInput
)

In [28]:
def fact_check(claim: str) -> str:
    # In a real scenario, you would call an actual fact-checking API here
    return "The claim appears to be true based on current information."

fact_check_tool = Tool(
    name="FactChecker",
    func=fact_check,
    description="Check the factual accuracy of a given claim"
)

In [29]:
def extract_keywords(text: str, num_keywords: int = 5) -> list:
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    sorted_items = sorted(enumerate(tfidf_matrix.toarray()[0]), key=lambda x: x[1], reverse=True)
    return [feature_names[i] for i, _ in sorted_items[:num_keywords]]

keyword_tool = Tool(
    name="KeywordExtractor",
    func=extract_keywords,
    description="Extract key terms from a given text"
)

In [30]:
def check_plagiarism(text: str) -> str:
    # In a real scenario, you would use a proper plagiarism checking service
    return "No significant plagiarism detected."

plagiarism_tool = Tool(
    name="PlagiarismChecker",
    func=check_plagiarism,
    description="Check text for potential plagiarism"
)

In [31]:
def save_to_file(content: str, filename: str) -> str:
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(content)
        return f"Content successfully saved to {filename}"
    except Exception as e:
        return f"Error saving content to file: {str(e)}"

save_file_tool = Tool(
    name="SaveToFile",
    func=save_to_file,
    description="Save content to a text file. Provide the content and the filename as arguments."
)

# Agents

In [32]:
researcher = Agent(
    role='Researcher',
    goal='Conduct thorough and unbiased research on given topics',
    backstory="""You are Dr. Alex Morgan, a seasoned research scientist with a Ph.D. in Interdisciplinary Studies. 
    Your academic journey has taken you through various fields, including natural sciences, social sciences, and humanities. 
    This diverse background has honed your ability to quickly grasp complex concepts across multiple disciplines.
    You've published in top-tier journals and are known for your meticulous approach to gathering and analyzing information. 
    Your colleagues often praise your ability to find connections between seemingly unrelated fields, leading to novel insights.
    You have a knack for using both traditional and cutting-edge research tools, always staying updated with the latest databases and research methodologies.""",
    tools=[search_tool, wikipedia_tool, web_scraper_tool, citation_tool, fact_check_tool, keyword_tool],
    verbose=True,
    llm=llm
)

In [33]:
writer = Agent(
    role='Writer',
    goal='Synthesize research findings into coherent, engaging, and informative reports',
    backstory="""You are Jamie Lee, a renowned science communicator and author with a Master's in Scientific Writing. 
    Your career began in journalism, where you covered everything from breaking news to in-depth features. 
    Over the years, you've developed a unique talent for translating complex scientific concepts into compelling narratives accessible to a broad audience.
    You've written best-selling popular science books and contribute regularly to prestigious publications like National Geographic and Scientific American. 
    Your writing style is known for its clarity, engagement, and ability to spark curiosity in readers. 
    You have a particular talent for using analogies and real-world examples to explain abstract or complex ideas.""",
    tools=[citation_tool, keyword_tool],
    llm=llm,
    verbose=True
)

In [34]:
fact_checker = Agent(
    role='Fact Checker',
    goal='Ensure the accuracy and reliability of all information in the report',
    backstory="""You are Dr. Sam Patel, a veteran fact-checker with a background in data science and library sciences. 
    Your career began at major news networks, where you developed a reputation for catching even the smallest inaccuracies. 
    You've since worked with scientific journals, book publishers, and online encyclopedias, honing your skills in verifying information across various fields.
    Your colleagues know you for your almost superhuman attention to detail and your vast network of expert contacts for verifying specialized information. 
    You're proficient in using advanced data analysis tools and have a deep understanding of source evaluation and fact-checking methodologies. 
    Your work has been crucial in maintaining the integrity of numerous high-profile publications.""",
    tools=[search_tool, wikipedia_tool, web_scraper_tool, fact_check_tool],
    llm=llm,
    verbose=True
)

In [35]:
editor = Agent(
    role='Editor',
    goal='Refine and polish articles for clarity, coherence, style, and maximum impact',
    backstory="""You are Morgan Chen, a senior editor with over two decades of experience in the publishing industry. 
    Your career has spanned academic journals, popular science magazines, and major publishing houses. 
    You have a reputation for turning good writing into great writing, with a particular talent for making complex topics accessible without losing depth or accuracy.
    Your editing philosophy focuses on enhancing the author's voice while ensuring clarity and engagement for the reader. 
    You have a keen eye for structure and flow, often reorganizing content to maximize its impact. 
    Your feedback is known for being constructive and insightful, helping writers to develop their skills while improving their current work. 
    You're also adept at tailoring content for different audiences, from peer-reviewed journals to mainstream media.""",
    tools=[keyword_tool, plagiarism_tool],
    llm=llm,
    verbose=True
)

In [14]:
survey_analyst = Agent(
    role='Survey Analyst',
    goal='Design, conduct, and analyze surveys to gather primary data on research topics',
    backstory="""You are Dr. Riley Zhang, a social scientist specializing in survey methodology and data analysis. 
    With a Ph.D. in Sociology and extensive experience in market research, you've honed your skills in crafting effective surveys that yield valuable insights. 
    Your expertise lies in designing unbiased questions, determining appropriate sample sizes, and employing advanced statistical techniques to interpret results. 
    You've worked on projects ranging from small-scale academic studies to large national polls, always ensuring the highest standards of data integrity and respondent privacy. 
    Your ability to translate complex survey data into actionable insights has made you a sought-after consultant in both academia and industry.""",
    tools=[],  # Add relevant tools for survey creation and analysis if needed
    llm=llm,
    verbose=True
)

In [15]:
file_manager = Agent(
    role='File Manager',
    goal='Save the final article to a text file',
    backstory="""You are the digital archivist responsible for preserving the final output of the research and writing process. Your role is crucial in ensuring that the valuable content created by the team is properly stored and easily accessible for future reference.""",
    tools=[save_file_tool],
    llm=llm,
    verbose=True
)

In [36]:
def create_tasks(topic):
    research_task = Task(
        description=f"""
        Conduct a comprehensive and multifaceted research on the topic: {topic}. 
        Your research should:
        1. Cover the fundamental aspects of the topic, including its history, current state, and future projections.
        2. Explore various perspectives and schools of thought related to the topic.
        3. Identify key figures, breakthrough moments, and pivotal studies in the field.
        4. Investigate real-world applications or implications of the subject matter.
        5. Analyze any controversies, debates, or unresolved questions within the field.
        6. Examine interconnections with other fields or areas of study.
        7. Gather relevant statistics, data, and case studies to support the findings.
        
        Use a combination of academic databases, reputable news sources, and expert opinions. 
        Leverage the search tool for broad information gathering, the Wikipedia tool for overview and references, 
        and the web scraper for extracting detailed information from specific websites.
        
        Compile your findings into a well-organized research summary, highlighting key points and potential areas for deeper exploration.
        """,
        agent=researcher,
        expected_output="A comprehensive research summary on the given topic."
    )

    writing_task = Task(
        description=f"""
        Based on the comprehensive research provided, write an engaging and informative article on {topic}. Your article should:
        1. Begin with a compelling introduction that hooks the reader and provides a clear overview of what the article will cover.
        2. Organize the content into logical sections with clear, informative headings.
        3. Explain complex concepts using analogies, metaphors, or real-world examples to make them accessible to a general audience.
        4. Incorporate relevant data, statistics, and expert quotes to support key points.
        5. Address different perspectives or debates within the field, presenting a balanced view where appropriate.
        6. Include specific examples, case studies, or anecdotes to illustrate important points and maintain reader engagement.
        7. Discuss the practical implications or applications of the topic in everyday life or various industries.
        8. Conclude with a summary of key takeaways and, if appropriate, future outlook or areas for further exploration.
        
        Aim for a tone that is informative yet conversational, catering to an educated general audience. 
        Ensure that the article flows logically from one point to the next, creating a coherent narrative throughout.
        """,
        agent=writer,
        expected_output="An engaging and informative article on the given topic."
    )

    fact_checking_task = Task(
        description=f"""
        Thoroughly verify the accuracy and reliability of all information presented in the article on {topic}. Your fact-checking process should:
        1. Cross-reference each significant fact or claim with at least two reputable sources.
        2. Verify the accuracy of all statistics, dates, names, and other specific data points.
        3. Check the credibility and current relevance of all sources cited in the article.
        4. Ensure that any theories or scientific concepts are explained accurately and reflect the current consensus in the field.
        5. Verify that quotes are accurate and not taken out of context.
        6. Check for any potential biases in the presentation of information and ensure a balanced perspective is maintained.
        7. Identify any areas where the information might be outdated or where more recent developments should be included.
        8. Flag any statements that are overly generalized or need more nuanced explanation.
        
        Use the search and Wikipedia tools to cross-check information, and the web scraper to verify details from original sources when necessary.
        Provide a detailed report of your findings, including:
        - A list of verified facts
        - Any corrections or clarifications needed
        - Suggestions for additional sources or information to strengthen the article
        - Areas where more research or expert input might be beneficial
        """,
        agent=fact_checker,
        expected_output="A detailed fact-checking report on the article."
    )

    editing_task = Task(
        description=f"""
        Review and refine the article on {topic}, elevating its quality, clarity, and impact. Your editing process should:
        1. Ensure the article has a strong, attention-grabbing opening and a satisfying, thought-provoking conclusion.
        2. Verify that the overall structure is logical and the flow of ideas is smooth and coherent.
        3. Check that each paragraph has a clear main idea and contributes to the overall narrative.
        4. Refine the language for clarity, concision, and engagement, eliminating jargon or overly complex phrasing.
        5. Enhance transitions between sections and ideas to improve overall readability.
        6. Ensure consistency in tone, style, and level of detail throughout the article.
        7. Verify that all technical terms are adequately explained for a general audience.
        8. Check for and eliminate any redundancies or unnecessary repetition.
        9. Ensure proper citation and attribution for all sources and quotes.
        10. Enhance the use of subheadings, bullet points, or other formatting elements to improve scannability and highlight key information.
        11. Suggest areas where additional examples, analogies, or visual elements could enhance understanding.
        12. Ensure the title and section headings are compelling and accurately reflect the content.
        
        Pay special attention to making the content accessible to a general educated audience while maintaining the depth and accuracy of the information.
        Provide a final edit of the article along with a summary of major changes and suggestions for any additional enhancements.
        """,
        agent=editor,
        expected_output="A refined and polished version of the article with editing notes."
    )

    # save_file_task = Task(
    #     description=f"""
    #     Save the final, edited article on {topic} to a text file.
    #     Your task should:
    #     1. Review the final article produced by the writing and editing process.
    #     2. Ensure all sections of the article are included and properly formatted.
    #     3. Use the SaveToFile tool to save the content to a file named '{topic.replace(' ', '_')}_article.txt'.
    #     4. Confirm that the file has been successfully saved and provide the file path.
    #     5. If there are any issues in saving the file, report the problem and suggest alternative solutions.
        
    #     This task is crucial for preserving the team's work and making it accessible for future reference or publication.
    #     """,
    #     agent=file_manager,
    #     expected_output = "A txt file saved with the contents of the research article in the memory"
    # )


    return [research_task, writing_task, fact_checking_task, editing_task]

In [37]:
def conduct_research(topic):
    tasks = create_tasks(topic)
    crew = Crew(
        agents=[researcher, survey_analyst, writer, fact_checker, editor],
        tasks=tasks,
        verbose=True,
        task_execution_timeout=200
    )
    return crew.kickoff()

In [ ]:
if __name__ == "__main__":
    research_topic = input("Enter a research topic: ")
    result = conduct_research(research_topic)
    print(result)
    
    # Inform the user about the saved file
    filename = f"{research_topic.replace(' ', '_')}_article.txt"
    if os.path.exists(filename):
        print(f"\nThe article has been saved to: {os.path.abspath(filename)}")
    else:
        print("\nNote: The article file was not found. There might have been an issue during the saving process.")

Enter a research topic:  Shivaji Maharaj


2024-08-29 19:05:04,992 - 139981923529344 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-29 19:05:05][DEBUG]: == Working Agent: Researcher
 [2024-08-29 19:05:05][INFO]: == Starting Task: 
        Conduct a comprehensive and multifaceted research on the topic: Shivaji Maharaj. 
        Your research should:
        1. Cover the fundamental aspects of the topic, including its history, current state, and future projections.
        2. Explore various perspectives and schools of thought related to the topic.
        3. Identify key figures, breakthrough moments, and pivotal studies in the field.
        4. Investigate real-world applications or implications of the subject matter.
        5. Analyze any controversies, debates, or unresolved questions within the field.
        6. Examine interconnections with other fields or areas of study.
        7. Gather relevant statistics, data, and case studies to support the findings.
        
        Use a combination of academic databases, reputable news sources, and expert opinions. 
        Leverage the search tool for broa